In [1]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import csv
import umap
import scanpy as sc

In [2]:
import h5py
from scipy.sparse import csr_matrix

def load_data(dataset,gene_percent_cells=0.01,peak_percent_cells=0.001,include_h5=True,
              preprocess=True,sampling=None,distance=1000000):
    
    data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)

    if sampling == 'geosketch':
        rna_adata = sc.read(os.path.join(data_dir,'{}.rna.sketch.h5ad'.format(dataset)))
        atac_adata = sc.read(os.path.join(data_dir,'{}.atac.sketch.h5ad'.format(dataset)))
    elif sampling == 'uniform':
        rna_adata = sc.read(os.path.join(data_dir,'{}.rna.uniform.h5ad'.format(dataset)))
        atac_adata = sc.read(os.path.join(data_dir,'{}.atac.uniform.h5ad'.format(dataset)))        
    else:
        rna_adata = sc.read(os.path.join(data_dir,'{}.rna.h5ad'.format(dataset)))
        atac_adata = sc.read(os.path.join(data_dir,'{}.atac.h5ad'.format(dataset)))

    if dataset == 'sci_car':
        data_dir = '/data/cb/alexwu/mm_finemap/data/sci_car'
        adata = sc.read(os.path.join(data_dir,'adata-hs.h5ad'))

        atac_adata.var['chr_no'] = ['chr' + n for n in atac_adata.var['chr_no']]
        atac_adata.var['start'] = adata.uns['atac.var'][:,2].astype(int)
        atac_adata.var['end'] = adata.uns['atac.var'][:,3].astype(int)

    if preprocess:
        # scale by maximum 
        # (rna already normalized by library size + log-transformed)
        X_max = rna_adata.X.max(0).toarray().squeeze()
        X_max[X_max == 0] = 1
        rna_adata.X = csr_matrix(rna_adata.X / X_max)

        # atac: normalize library size + log transformation
        sc.pp.normalize_total(atac_adata,target_sum=1e4)
        sc.pp.log1p(atac_adata)

    if include_h5:
        data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)
        with h5py.File(os.path.join(data_dir,'dataset.h5'),'r') as f:
            print(f.keys())
            data_df = pd.DataFrame()
            for k in ['chr_no','eqtl','hic','dist','atac_id','gene','corr','group_corr','gene_percent_cells',
                      'peak_percent_cells','hg38tohg19_eqtl','eqtl.new','corr_bin','group_corr_bin',
                      'hic_0hr','hic_1hr','hic_4hr','eqtl.q','hic_4hr.expectedBL','eqtl.all','hic.new2',
                      'hic.100kb','ABC.score']: #,
#                      'hic_0hr.expectedBL','hic_1hr.expectedBL','hic_4hr.expectedBL.new','hic_0hr.expectedBL.new',
#                      'hic_4hr.expectedBL.new2','hic_12hr.expectedBL','hic_4hr.expectedBL.full']:
                if k in f.keys():
                    data_df[k] = f[k][:]

        small_df = data_df[abs(data_df['dist']) < distance]

        # small_df = small_df[~np.isnan(small_df['group_corr'])]
        small_df = small_df[small_df['gene_percent_cells'] > gene_percent_cells]
        small_df = small_df[small_df['peak_percent_cells'] > peak_percent_cells]

        small_df.index = [(atac_idx,gene) for atac_idx,gene in small_df[['atac_id','gene']].values]
    
        return rna_adata,atac_adata,small_df

    else:
        
        return rna_adata,atac_adata

In [3]:
from scipy.stats import rankdata

def load_results(dataset,eval_df,ensemble=False,gene_percent_cells=0.01,peak_percent_cells=0.001):
    
    n_layers = 10
    n_neighbors = 15
    mode = 'lr'

    eval_df['ABC.score'] = eval_df['ABC.score'].replace(np.nan, 0)
    eval_df['ABC.score'] = eval_df['ABC.score'].values #.astype(bool).astype(float)

    eval_df['corr'] = eval_df['corr'].replace(np.nan, 0)
    eval_df['group_corr'] = eval_df['group_corr'].replace(np.nan, 0)

    data_dir = '/data/cb/alexwu/mm_finemap/results/tests_nn/{}'.format(dataset)
        
    method = 'graph'
    for trial_no in range(1,6):

        if trial_no == 1 and dataset in ['human_cortex_multiome_lineage','share_seq_more']:
            data_dir = '/data/cb/alexwu/mm_finemap/results/tests_nn/{}'.format(dataset)
        else:
            data_dir = '/data/cb/alexwu/mm_finemap/results/tests_nn/{}_{}'.format(dataset,trial_no)

        if dataset == 'human_cortex_multiome_lineage':
            file_name = 'graph.uniform.250kb.trial{}.all.max_scale.mseloss.{}layers.nn{}.statistics.txt'.format(trial_no,n_layers,n_neighbors)
        elif dataset == 'share_seq_more':
            file_name = 'graph.uniform.trial{}.all.max_scale.mseloss.{}layers.nn{}.statistics.txt'.format(trial_no,n_layers,n_neighbors)
        else:
            file_name = 'graph.uniform.1Mb.trial{}.all.max_scale.mseloss.{}layers.nn{}.statistics.txt'.format(trial_no,n_layers,n_neighbors)

        scores = pd.read_csv(os.path.join(data_dir,file_name),sep='\t').values[:,2].astype(float)
        scores[np.isinf(scores)] = 10**10
        eval_df['{}.trial{}'.format(method,trial_no)] = scores


        if trial_no == 1:
            data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)
        else:
            data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}_{}'.format(dataset,trial_no)

        # correlations
        with h5py.File(os.path.join(data_dir,'dataset.h5'),'r') as f:
            data_df = pd.DataFrame()
            for k in ['dist','corr','group_corr','gene_percent_cells','peak_percent_cells']:
                if k in f.keys():
                    data_df[k] = f[k][:]

        small_df = data_df[abs(data_df['dist']) < distance]

        # small_df = small_df[~np.isnan(small_df['group_corr'])]
        small_df = small_df[small_df['gene_percent_cells'] > gene_percent_cells]
        small_df = small_df[small_df['peak_percent_cells'] > peak_percent_cells]

        eval_df['corr.trial{}'.format(trial_no)] = small_df['corr'].replace(np.nan, 0).values
        eval_df['group_corr.trial{}'.format(trial_no)] = small_df['group_corr'].replace(np.nan, 0).values

    if ensemble:
                
        for method in ['graph','corr','group_corr']:
            eval_df['{}.ranks.mean'.format(method)] = np.array([rankdata(abs(eval_df['{}.trial{}'.format(method,trial_no)].values),
                                                                method='average') for trial_no in range(1,6)]).mean(0)

        distance_ranks = rankdata(-abs(eval_df['dist'].values),method='average')
        eval_df['dist_ranks'] = distance_ranks

        for feature in ['graph.ranks.mean','group_corr.ranks.mean','corr.ranks.mean']:
            eval_df[feature + '-dist'] = (distance_ranks + eval_df[feature].values)/2
            
    return eval_df

In [4]:
import math

def round_sigfigs(number,significant_digits=3):
    return round(number, significant_digits - int(math.floor(math.log10(abs(number)))) - 1)

def num_sigfigs(number):
    return len(str(number).split('.')[1].strip('0'))
    
def create_plot(results_df,metric):
    
    plt.figure(figsize=(3,5))
    sns.barplot(x='method',y=metric,data=results_df,capsize=0.1)
    plt.ylim(0,results_df[metric].max()*1.02)
    plt.xlabel('Method',fontsize=16)
    plt.ylabel(metric.upper(),fontsize=16)
    plt.yticks(fontsize=14)
    plt.xticks(rotation=90)

    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=True) # labels along the bottom edge are off
    
#     df_dict = {n: df for n,df in results_df.groupby(by='method',sort=False)}
    
#     for n in method_list:
#         df = df_dict[n]
#         mean = round_sigfigs(df[metric].mean())
#         if num_sigfigs(mean) < 3:
#             mean = str(mean) + '0'
#         std = round(df[metric].std(),len(str(mean).split('.')[1]))
        
#         print(n,mean,std)
    
#     print('----------')
    

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
from schema import SchemaQP
from scipy.stats import spearmanr
from collections import Counter

def preprocess(rna_adata,atac_adata):
    
    rna_adata.obs['n_counts'] = rna_adata.X.sum(1).astype(int)
    atac_adata.obs['n_counts'] = atac_adata.X.sum(1).astype(int)

    sc.pp.normalize_total(rna_adata,target_sum=1e5)
    sc.pp.log1p(rna_adata)
    
    rna_adata.var['gene_percent_cells'] = (rna_adata.X != 0).toarray().squeeze().mean(0)
    atac_adata.var['peak_percent_cells'] = (atac_adata.X != 0).toarray().squeeze().mean(0)

    # for genes represented via multiple transcripts, retain gene represented in most # of cells
    counts = Counter(rna_adata.var['symbol'].values)

    inds2keep = list(range(rna_adata.shape[1]))

    for gene,v in counts.items():
        if v > 1:
            inds = np.where(rna_adata.var['symbol'] == gene)[0]
            max_ind = inds[np.argmax(rna_adata.var.iloc[inds]['gene_percent_cells'].values)]
            for i in inds:
                if i != max_ind:
                    inds2keep.remove(i)

    rna_adata = rna_adata[:,inds2keep].copy()
    rna_adata.var.index = rna_adata.var['symbol'].values

    # remove genes,peaks in unlocalized sequences
    gene_inds2keep = [i for i,n in enumerate(rna_adata.var['chr_no'].values) if 'chr' in str(n)]
    atac_inds2keep = [i for i,n in enumerate(atac_adata.var['chr_no'].values) if 'chr' in str(n)]

    rna_adata = rna_adata[:,gene_inds2keep]
    atac_adata = atac_adata[:,atac_inds2keep]

    atac_adata.var.index = list(map(str,range(atac_adata.shape[1])))
    
    return rna_adata,atac_adata

def schema_representations(rna_adata,atac_adata,schema_reference='rna',other_modalities=None):

    # LEARN JOINT REPRESENTATIONS
    rna_adata_copy = rna_adata.copy()
    atac_adata_copy = atac_adata.copy()

    sc.pp.filter_genes(rna_adata_copy,min_cells=int(0.001*rna_adata.shape[0]))
    sc.pp.filter_genes(atac_adata_copy,min_cells=int(0.001*rna_adata.shape[0]))

    sc.pp.highly_variable_genes(rna_adata_copy,n_top_genes=2000)
    rna_adata_copy = rna_adata_copy[:,rna_adata_copy.var['highly_variable']]
    sc.pp.scale(rna_adata_copy)    
    
    tfidf = TfidfTransformer()
    idf = tfidf.fit_transform(atac_adata_copy.X)

    atac_adata_copy.X = idf

    sc.tl.pca(rna_adata_copy,n_comps=50)
    sc.tl.pca(atac_adata_copy,n_comps=50)

    rna_adata.obsm['X_pca'] = rna_adata_copy.obsm['X_pca']
    atac_adata.obsm['X_pca'] = atac_adata_copy.obsm['X_pca']

    atac_pca_inds2keep = []
    rna_pca_inds2keep = []
    for i in range(50):
        rho,p = spearmanr(atac_adata_copy.obsm['X_pca'][:,i],atac_adata_copy.obs['n_counts'])
        if rho < 0.9:
            atac_pca_inds2keep.append(i)

        rho,p = spearmanr(rna_adata_copy.obsm['X_pca'][:,i],rna_adata_copy.obs['n_counts'])
        if rho < 0.9:
            rna_pca_inds2keep.append(i)
            
    n_components = 50
    model = SchemaQP(0.99,params = {"decomposition_model": "pca", "num_top_components": n_components},\
                    mode = 'scale')

    if schema_reference == 'rna':
        secondary_modalities = [atac_adata.obsm['X_pca'][:,atac_pca_inds2keep]]
        if other_modalities:
            secondary_modalities.extend(other_modalities)
            
        W = model.fit_transform(rna_adata.obsm['X_pca'][:,rna_pca_inds2keep], # primary dataset
                                   secondary_modalities, # just one secondary dataset
                                   ['feature_vector'], # has labels, i.e., is a categorical datatype
                                   [1]
                                  )
        
    elif schema_reference == 'atac':
        secondary_modalities = [rna_adata.obsm['X_pca'][:,rna_pca_inds2keep]]
        if other_modalities:
            secondary_modalities.extend(other_modalities)
            
        W = model.fit_transform(atac_adata.obsm['X_pca'][:,atac_pca_inds2keep], # primary dataset
                                   secondary_modalities, # just one secondary dataset
                                   ['feature_vector'], # has labels, i.e., is a categorical datatype
                                   [1]
                                  )

    rna_adata.obsm['X_schema'] = W
    atac_adata.obsm['X_schema'] = W
    
def set_dpt_root(rna_adata,start_marker_gene):
    
    rna_adata.uns['iroot'] = np.argsort(-rna_adata[:,start_marker_gene].X.toarray().squeeze())[0]

In [6]:
# from geosketch import gs

# def apply_geosketch(dataset,rna_adata,atac_adata,start_marker_gene,N=5000):
    
#     data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)

#     sketch_index = gs(rna_adata.obsm['X_schema'], N, replace=False)
    
#     rna_adata = rna_adata[sketch_index]
#     atac_adata = atac_adata[sketch_index]

#     rna_adata.uns['iroot'] = np.argsort(-rna_adata[:,start_marker_gene].X.toarray().squeeze())[0]

#     atac_adata.write_h5ad(os.path.join(data_dir,'{}.atac.sketch.h5ad'.format(dataset)))
#     rna_adata.write_h5ad(os.path.join(data_dir,'{}.rna.sketch.h5ad'.format(dataset)))

def apply_uniform_sampling(dataset,rna_adata,atac_adata,start_marker_gene,N=5000,seed=1):
    
    np.random.seed(seed)
    
    data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)

    sketch_index = np.random.choice(range(rna_adata.shape[0]),size=N,replace=False)
    
    rna_adata_sample = rna_adata[sketch_index].copy()
    atac_adata_sample = atac_adata[sketch_index].copy()

    rna_adata_sample.uns['iroot'] = np.argsort(-rna_adata_sample[:,start_marker_gene].X.toarray().squeeze())[0]

    atac_adata_sample.write_h5ad(os.path.join(data_dir,'{}.atac.uniform.h5ad'.format(dataset)))
    rna_adata_sample.write_h5ad(os.path.join(data_dir,'{}.rna.uniform.h5ad'.format(dataset)))


In [7]:
def calculate_correlations(data_df,rna_adata,atac_adata,corr_key):
    
    if 'atac_idx' in data_df.keys() and 'atac_id' not in data_df.keys():
        data_df['atac_id'] = data_df['atac_idx'].values
        
    corr_dict = {}
    for chr_no in sorted(list(set(data_df['chr_no'].values))):
        print(chr_no)
        filtered_data_df = data_df[data_df['chr_no'] == chr_no]

        gene_inds = sorted(list(set(filtered_data_df['gene'].values)))
        atac_inds = list(map(str,sorted(list(set(filtered_data_df['atac_id'].values.astype(int))))))

        filtered_rna_X = rna_adata[:,gene_inds].X.toarray()
        filtered_atac_X = atac_adata[:,atac_inds].X.toarray()

        rho = corr2_coeff(filtered_rna_X.T,filtered_atac_X.T)
        df = pd.DataFrame(rho,index=gene_inds,columns=atac_inds)

        for gene,atac_idx in filtered_data_df[['gene','atac_id']].values:
            corr_dict[(gene,atac_idx)] = df.loc[gene,str(atac_idx)]
            
    corr_list = []
    for gene,atac_idx in data_df[['gene','atac_id']].values:
        corr_list.append(corr_dict[(gene,atac_idx)])

    data_df[corr_key] = corr_list

    return data_df

def create_metacells(rna_adata,atac_adata):
    
    group_rna_X = []
    group_atac_X = []
    rna_X = rna_adata.X.toarray()
    rna_X = (((np.exp(rna_X)-1)/1e5).T*rna_adata.obs['n_counts'].values).T # raw counts

    for i in range(rna_adata.shape[0]):
        inds = np.nonzero(rna_adata.obsp['distances'][i])[1]
        inds = np.array(list(inds) + [i])
        group_rna_X.append(rna_X[inds].sum(0))
        group_atac_X.append(atac_adata[inds].X.sum(0))

        if i % 100 == 0:
            print(i)

    group_rna_X = np.array(group_rna_X).squeeze()
    group_atac_X = np.array(group_atac_X).squeeze()
    
    group_rna_adata = rna_adata.copy()
    group_atac_adata = atac_adata.copy()

    group_rna_adata.X = group_rna_X #.copy()
    group_atac_adata.X = group_atac_X #.copy()
    
    sc.pp.normalize_total(group_rna_adata,target_sum=1e5)
    sc.pp.log1p(group_rna_adata)

    sc.pp.normalize_total(group_atac_adata,target_sum=1e4)
    sc.pp.log1p(group_atac_adata)
    
    return group_rna_adata,group_atac_adata

def perform_correlation_calculations(data_df,rna_adata,atac_adata,dataset,mode):
    
    rna_adata_copy = rna_adata.copy()
    atac_adata_copy = atac_adata.copy()
    
    if mode == 'corr':
        sc.pp.normalize_total(atac_adata_copy,target_sum=1e4)
        sc.pp.log1p(atac_adata_copy)
        
    elif mode == 'group_corr':
        sc.pp.neighbors(rna_adata_copy,use_rep='X_schema',n_neighbors=50)
        rna_adata_copy,atac_adata_copy = create_metacells(rna_adata_copy,atac_adata_copy)

    corr_key = mode
    data_df = calculate_correlations(data_df,rna_adata_copy,atac_adata_copy,corr_key)

    save_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)

    with h5py.File(os.path.join(save_dir,'dataset.h5'),'a') as f:
        if corr_key in f.keys():
            del f[corr_key]
        f.create_dataset(corr_key,data=data_df[corr_key].values,dtype=float)

In [8]:
def calculate_gene_peak_percent_cells(data_df,rna_adata,atac_adata,dataset):
    gene_percent_cells = {g: percent for g,percent \
                          in zip(*[rna_adata.var.index.values,np.array((rna_adata.X != 0).mean(0)).squeeze()])}
    peak_percent_cells = {atac_idx: percent for atac_idx,percent \
                          in zip(*[atac_adata.var.index.values,np.array((atac_adata.X != 0).mean(0)).squeeze()])}

    atac_percent = []
    rna_percent = []
    for gene,atac_idx in data_df[['gene','atac_id']].values:
        atac_percent.append(peak_percent_cells[str(atac_idx)])
        rna_percent.append(gene_percent_cells[gene])

    data_df['gene_percent_cells'] = rna_percent
    data_df['peak_percent_cells'] = atac_percent

    import h5py

    save_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)

    with h5py.File(os.path.join(save_dir,'dataset.h5'),'a') as f:
        for key in ['gene_percent_cells','peak_percent_cells']:
            if k in f.keys():
                del f[key]
            f.create_dataset(key,data=data_df[key].values,dtype=float)

In [9]:
def write_rna_atac_indices(dataset,gene_percent_cells=0.01,peak_percent_cells=0.001,dist=1000000):

    data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)
    with h5py.File(os.path.join(data_dir,'dataset.h5'),'r') as f:
        data_df = pd.DataFrame()
        for k in ['atac_id','gene','dist','gene_percent_cells','peak_percent_cells']:
            if k in f.keys():
                data_df[k] = f[k][:]

    small_df = data_df[abs(data_df['dist']) < dist]

    # small_df = small_df[~np.isnan(small_df['group_corr'])]
    small_df = small_df[small_df['gene_percent_cells'] > gene_percent_cells]
    small_df = small_df[small_df['peak_percent_cells'] > peak_percent_cells]

    print(small_df.shape)

    rna_adata = sc.read(os.path.join(data_dir,'{}.rna.h5ad'.format(dataset)))
    atac_adata = sc.read(os.path.join(data_dir,'{}.atac.h5ad'.format(dataset)))

    gene_idx_dict = {g:i for i,g in enumerate(rna_adata.var.index.values)}

    data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)

    gene_idx = np.array([gene_idx_dict[g] for g in small_df['gene'].values])
    np.savetxt(os.path.join(data_dir,'atac_idx.txt'),small_df['atac_id'].values,fmt='%i')
    np.savetxt(os.path.join(data_dir,'rna_idx.txt'),gene_idx,fmt='%i')

In [10]:
def corr2_coeff(A, B):
    # Rowwise mean of input arrays & subtract from input arrays themeselves
    A_mA = A - A.mean(1)[:, None]
    B_mB = B - B.mean(1)[:, None]

    # Sum of squares across rows
    ssA = (A_mA**2).sum(1)
    ssB = (B_mB**2).sum(1)

    # Finally get corr coeff
    return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))

## Both Branches

In [11]:
dataset = 'share_seq'

rna_adata,atac_adata = load_data(dataset,sampling=None,peak_percent_cells=0.001,include_h5=False,preprocess=False)

In [12]:
data_dir = '/data/cb/alexwu/mm_finemap/data/share_seq'

cell_type_df = pd.read_csv(os.path.join(data_dir,'GSM4156597_skin_celltype.txt.gz'),sep='\t')
cell_type_df.index = cell_type_df['rna.bc'].values

rna_adata.obs['cell_type'] = cell_type_df.loc[rna_adata.obs.index.values]['celltype'].values
atac_adata.obs['cell_type'] = cell_type_df.loc[rna_adata.obs.index.values]['celltype'].values

In [13]:
cell_types2keep = ["TAC-1","TAC-2","Hair Shaft-cuticle.cortex","IRS","Medulla"]
rna_adata = rna_adata[rna_adata.obs["cell_type"].isin(cell_types2keep)]
atac_adata = atac_adata[atac_adata.obs["cell_type"].isin(cell_types2keep)]

In [15]:
start_marker_gene = 'Dlx3'

idx = np.argsort(-rna_adata[:,start_marker_gene].X.toarray().squeeze())[0]
rna_adata.uns['iroot'] = idx

/data/cb/alexwu/anaconda3/envs/lagnet/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


*Uniform Sampling*

In [18]:
start_marker_gene = 'Dlx3'

for k in range(1,6):
    print(k)
    dataset = 'share_seq_both_branches_{}'.format(k)
    apply_uniform_sampling(dataset,rna_adata,atac_adata,start_marker_gene,N=5000,seed=k)

1
2
3
4
5


### Calculate Correlations

In [11]:
for dataset in ['share_seq_both_branches_{}'.format(trial_no) for trial_no in range(2,6)]:

    print(dataset)
    
    data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)

    rna_adata = sc.read(os.path.join(data_dir,'{}.rna.uniform.h5ad'.format(dataset)))
    atac_adata = sc.read(os.path.join(data_dir,'{}.atac.uniform.h5ad'.format(dataset)))
    atac_adata.var.index = atac_adata.var.index.values.astype(int).astype(str)

    data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)
    with h5py.File(os.path.join(data_dir,'dataset.h5'),'r') as f:
        data_df = pd.DataFrame()
        for k in ['chr_no','eqtl','hic','dist','atac_id','gene','corr']:
            if k in f.keys():
                data_df[k] = f[k][:]
                
    data_df['atac_id'] = data_df['atac_id'].values.astype(int).astype(str)
    data_df['gene'] = data_df['gene'].values.astype(str)

    mode = 'corr'
    perform_correlation_calculations(data_df,rna_adata,atac_adata,dataset,mode)

    mode = 'group_corr'
    perform_correlation_calculations(data_df,rna_adata,atac_adata,dataset,mode)
    

share_seq_both_branches_2
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
b'chr1'


/tmp/ipykernel_355509/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
b'chr1'


/tmp/ipykernel_355509/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
share_seq_both_branches_3
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
b'chr1'
b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
b'chr1'


...

b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
share_seq_both_branches_4
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
b'chr1'
b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
b'chr1'


/tmp/ipykernel_355509/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
share_seq_both_branches_5
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
b'chr1'
b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
b'chr1'


/tmp/ipykernel_355509/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'


### Writing RNA + ATAC Indices

In [ ]:
for trial_no in range(1,6):
    
    dataset = 'share_seq_both_branches_{}'.format(trial_no)
    write_rna_atac_indices(dataset,peak_percent_cells=0.001,dist=500000)

## IRS Trajectory

In [11]:
new_dataset_name = "share_seq_irs"
cell_types2keep = ["TAC-1","TAC-2","IRS"]

In [12]:
dataset = 'share_seq'

rna_adata,atac_adata = load_data(dataset,sampling=None,peak_percent_cells=0.001,include_h5=False,preprocess=False)

data_dir = '/data/cb/alexwu/mm_finemap/data/share_seq'

cell_type_df = pd.read_csv(os.path.join(data_dir,'GSM4156597_skin_celltype.txt.gz'),sep='\t')
cell_type_df.index = cell_type_df['rna.bc'].values

rna_adata.obs['cell_type'] = cell_type_df.loc[rna_adata.obs.index.values]['celltype'].values
atac_adata.obs['cell_type'] = cell_type_df.loc[rna_adata.obs.index.values]['celltype'].values

In [16]:
rna_adata = rna_adata[rna_adata.obs["cell_type"].isin(cell_types2keep)]


In [35]:
rna_adata = rna_adata[rna_adata.obs["cell_type"].isin(cell_types2keep)]
atac_adata = atac_adata[atac_adata.obs["cell_type"].isin(cell_types2keep)]

start_marker_gene = 'Dlx3'

idx = np.argsort(-rna_adata[:,start_marker_gene].X.toarray().squeeze())[0]
rna_adata.uns['iroot'] = idx

start_marker_gene = 'Dlx3'

for k in range(1,6):
    print(k)
    dataset = '{}_{}'.format(new_dataset_name,k)
    apply_uniform_sampling(dataset,rna_adata,atac_adata,start_marker_gene,N=5000,seed=k)

/data/cb/alexwu/anaconda3/envs/lagnet/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


1
2
3
4
5


In [37]:
# for trial_no in range(1,6):
    
#     dataset = '{}_{}'.format(new_dataset_name,trial_no)
#     write_rna_atac_indices(dataset,peak_percent_cells=0.001,dist=500000)

In [13]:
for dataset in ['{}_{}'.format(new_dataset_name,trial_no) for trial_no in range(5,6)]:

    print(dataset)
    
    data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)

    rna_adata = sc.read(os.path.join(data_dir,'{}.rna.uniform.h5ad'.format(dataset)))
    atac_adata = sc.read(os.path.join(data_dir,'{}.atac.uniform.h5ad'.format(dataset)))
    atac_adata.var.index = atac_adata.var.index.values.astype(int).astype(str)

    data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)
    with h5py.File(os.path.join(data_dir,'dataset.h5'),'r') as f:
        data_df = pd.DataFrame()
        for k in ['chr_no','eqtl','hic','dist','atac_id','gene','corr']:
            if k in f.keys():
                data_df[k] = f[k][:]
                
    data_df['atac_id'] = data_df['atac_id'].values.astype(int).astype(str)
    data_df['gene'] = data_df['gene'].values.astype(str)

    mode = 'corr'
    perform_correlation_calculations(data_df,rna_adata,atac_adata,dataset,mode)

    mode = 'group_corr'
    perform_correlation_calculations(data_df,rna_adata,atac_adata,dataset,mode)


share_seq_irs_5
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
b'chr1'


/tmp/ipykernel_1652184/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
b'chr1'


/tmp/ipykernel_1652184/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'


## Medulla  Trajectory

In [21]:
new_dataset_name = "share_seq_medulla"
cell_types2keep = ["TAC-1","TAC-2","Medulla"]

In [24]:
dataset = 'share_seq'

rna_adata,atac_adata = load_data(dataset,sampling=None,peak_percent_cells=0.001,include_h5=False,preprocess=False)

data_dir = '/data/cb/alexwu/mm_finemap/data/share_seq'

cell_type_df = pd.read_csv(os.path.join(data_dir,'GSM4156597_skin_celltype.txt.gz'),sep='\t')
cell_type_df.index = cell_type_df['rna.bc'].values

print(rna_adata.shape)
rna_adata.obs['cell_type'] = cell_type_df.loc[rna_adata.obs.index.values]['celltype'].values
atac_adata.obs['cell_type'] = cell_type_df.loc[rna_adata.obs.index.values]['celltype'].values

rna_adata = rna_adata[rna_adata.obs["cell_type"].isin(cell_types2keep)]
atac_adata = atac_adata[atac_adata.obs["cell_type"].isin(cell_types2keep)]
print(rna_adata.shape)

start_marker_gene = 'Dlx3'

idx = np.argsort(-rna_adata[:,start_marker_gene].X.toarray().squeeze())[0]
rna_adata.uns['iroot'] = idx

start_marker_gene = 'Dlx3'

for k in range(1,6):
    print(k)
    dataset = '{}_{}'.format(new_dataset_name,k)
    apply_uniform_sampling(dataset,rna_adata,atac_adata,start_marker_gene,N=5000,seed=k)

(34774, 23296)
(5359, 23296)
1
2
3
4
5


In [ ]:
# for trial_no in range(1,6):
    
#     dataset = '{}_{}'.format(new_dataset_name,trial_no)
#     write_rna_atac_indices(dataset,peak_percent_cells=0.001,dist=500000)

In [15]:
for dataset in ['{}_{}'.format(new_dataset_name,trial_no) for trial_no in range(1,6)]:

    print(dataset)
    
    data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)

    rna_adata = sc.read(os.path.join(data_dir,'{}.rna.uniform.h5ad'.format(dataset)))
    atac_adata = sc.read(os.path.join(data_dir,'{}.atac.uniform.h5ad'.format(dataset)))
    atac_adata.var.index = atac_adata.var.index.values.astype(int).astype(str)

    data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)
    with h5py.File(os.path.join(data_dir,'dataset.h5'),'r') as f:
        data_df = pd.DataFrame()
        for k in ['chr_no','eqtl','hic','dist','atac_id','gene','corr']:
            if k in f.keys():
                data_df[k] = f[k][:]
                
    data_df['atac_id'] = data_df['atac_id'].values.astype(int).astype(str)
    data_df['gene'] = data_df['gene'].values.astype(str)

    mode = 'corr'
    perform_correlation_calculations(data_df,rna_adata,atac_adata,dataset,mode)

    mode = 'group_corr'
    perform_correlation_calculations(data_df,rna_adata,atac_adata,dataset,mode)
    
# for trial_no in range(1,6):
    
#     dataset = '{}_{}'.format(new_dataset_name,trial_no)
#     write_rna_atac_indices(dataset,peak_percent_cells=0.001,dist=500000)

share_seq_medulla_1
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
b'chr1'


/tmp/ipykernel_1652184/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
b'chr1'


/tmp/ipykernel_1652184/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
share_seq_medulla_2
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
b'chr1'
b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
b'chr1'


...

b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
share_seq_medulla_4
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
b'chr1'
b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
b'chr1'


/tmp/ipykernel_1652184/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
share_seq_medulla_5
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
b'chr1'
b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
b'chr1'


/tmp/ipykernel_1652184/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'


## Cortex  Trajectory

In [25]:
new_dataset_name = "share_seq_cortex"
cell_types2keep = ["TAC-1","TAC-2","Hair Shaft-cuticle.cortex"]

In [26]:
dataset = 'share_seq'

rna_adata,atac_adata = load_data(dataset,sampling=None,peak_percent_cells=0.001,include_h5=False,preprocess=False)

data_dir = '/data/cb/alexwu/mm_finemap/data/share_seq'

cell_type_df = pd.read_csv(os.path.join(data_dir,'GSM4156597_skin_celltype.txt.gz'),sep='\t')
cell_type_df.index = cell_type_df['rna.bc'].values

print(rna_adata.shape)
rna_adata.obs['cell_type'] = cell_type_df.loc[rna_adata.obs.index.values]['celltype'].values
atac_adata.obs['cell_type'] = cell_type_df.loc[rna_adata.obs.index.values]['celltype'].values

rna_adata = rna_adata[rna_adata.obs["cell_type"].isin(cell_types2keep)]
atac_adata = atac_adata[atac_adata.obs["cell_type"].isin(cell_types2keep)]
print(rna_adata.shape)

start_marker_gene = 'Dlx3'

idx = np.argsort(-rna_adata[:,start_marker_gene].X.toarray().squeeze())[0]
rna_adata.uns['iroot'] = idx

start_marker_gene = 'Dlx3'

for k in range(1,6):
    print(k)
    dataset = '{}_{}'.format(new_dataset_name,k)
    apply_uniform_sampling(dataset,rna_adata,atac_adata,start_marker_gene,N=5000,seed=k)

(34774, 23296)
(5544, 23296)
1
2
3
4
5


In [17]:
for dataset in ['{}_{}'.format(new_dataset_name,trial_no) for trial_no in range(1,6)]:

    print(dataset)
    
    data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)

    rna_adata = sc.read(os.path.join(data_dir,'{}.rna.uniform.h5ad'.format(dataset)))
    atac_adata = sc.read(os.path.join(data_dir,'{}.atac.uniform.h5ad'.format(dataset)))
    atac_adata.var.index = atac_adata.var.index.values.astype(int).astype(str)

    data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)
    with h5py.File(os.path.join(data_dir,'dataset.h5'),'r') as f:
        data_df = pd.DataFrame()
        for k in ['chr_no','eqtl','hic','dist','atac_id','gene','corr']:
            if k in f.keys():
                data_df[k] = f[k][:]
                
    data_df['atac_id'] = data_df['atac_id'].values.astype(int).astype(str)
    data_df['gene'] = data_df['gene'].values.astype(str)

    mode = 'corr'
    perform_correlation_calculations(data_df,rna_adata,atac_adata,dataset,mode)

    mode = 'group_corr'
    perform_correlation_calculations(data_df,rna_adata,atac_adata,dataset,mode)
    

# for trial_no in range(1,6):
    
#     dataset = '{}_{}'.format(new_dataset_name,trial_no)
#     write_rna_atac_indices(dataset,peak_percent_cells=0.001,dist=500000)

share_seq_cortex_1
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
b'chr1'


/tmp/ipykernel_1652184/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
b'chr1'


/tmp/ipykernel_1652184/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
share_seq_cortex_2
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
b'chr1'
b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
b'chr1'


...

b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
share_seq_cortex_4
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
b'chr1'
b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
b'chr1'


/tmp/ipykernel_1652184/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
share_seq_cortex_5
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
b'chr1'
b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
b'chr1'


/tmp/ipykernel_1652184/3788103053.py:11: RuntimeWarning: invalid value encountered in divide
  return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))


b'chr10'
b'chr11'
b'chr12'
b'chr13'
b'chr14'
b'chr15'
b'chr16'
b'chr17'
b'chr18'
b'chr19'
b'chr2'
b'chr3'
b'chr4'
b'chr5'
b'chr6'
b'chr7'
b'chr8'
b'chr9'
b'chrX'


## Results

In [13]:
dataset = 'share_seq_more'

distance = 1000000
gene_percent_cells = 0.01
peak_percent_cells = 0.001

# sketch = 'glial' not in dataset
rna_adata,atac_adata,eval_df = load_data(dataset,gene_percent_cells=gene_percent_cells,
                                          peak_percent_cells=peak_percent_cells,
                                          include_h5=True,sampling='uniform',distance=distance)

eval_df = load_results(dataset,eval_df,ensemble=True)
eval_df['abs_dist'] = abs(eval_df['dist'])

Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
Only considering the two last: ['.uniform', '.h5ad'].
<KeysViewHDF5 ['ABC.score', 'atac_id', 'chr_no', 'corr', 'corr.sketch', 'corr_bin', 'dist', 'eqtl.all', 'eqtl.fdr', 'eqtl.q', 'gene', 'gene_percent_cells', 'group_corr', 'group_corr.sketch', 'hic', 'hic.new', 'hic.new2', 'peak_percent_cells']>


### All Branches: Granger Causality

In [15]:
# load standard Granger + GVAR results
dataset = 'share_seq_both_branches'

for trial_no in range(1,6):
    df = pd.read_csv(os.path.join(data_dir,"{}_{}".format(dataset,trial_no),"{}_{}.standard_granger.csv".format(dataset,trial_no)))
    eval_df["granger.trial{}".format(trial_no)] = -np.log10(df["granger_p"].values)

In [ ]:
# load standard Granger + GVAR results
dataset = 'share_seq_more'

data_dir = "/data/cb/alexwu/mm_finemap/results/tests_nn"
df = pd.read_csv(os.path.join(data_dir,dataset,"{}.GVAR.csv".format(dataset,trial_no)))
p = [float(n.strip("[]")) for n in df["granger_p"].values]
eval_df["gvar.trial{}".format(1)] = -np.log10(df["granger_p"].values)
for trial_no in range(2,6):
    df = pd.read_csv(os.path.join(data_dir,"{}_{}".format(dataset,trial_no),"{}_{}.GVAR.csv".format(dataset,trial_no)))
    p = [float(n.strip("[]")) for n in df["granger_p"].values]
    eval_df["gvar.trial{}".format(trial_no)] = -np.log10(p)

In [13]:
dataset = "share_seq_both_branches"

n_layers = 10
n_neighbors = 15
mode = 'lr'

eval_df = small_df.copy()
eval_df['ABC.score'] = eval_df['ABC.score'].replace(np.nan, 0)
eval_df['ABC.score'] = eval_df['ABC.score'].values #.astype(bool).astype(float)

eval_df['corr'] = eval_df['corr'].replace(np.nan, 0)
eval_df['group_corr'] = eval_df['group_corr'].replace(np.nan, 0)

data_dir = '/data/cb/alexwu/mm_finemap/results/tests_nn/{}'.format(dataset)
method = 'graph'
for trial_no in range(1,6):

    print('Trial {}'.format(trial_no))
    if trial_no == 1:
        data_dir = '/data/cb/alexwu/mm_finemap/results/tests_nn/{}'.format(dataset)

    else:
        data_dir = '/data/cb/alexwu/mm_finemap/results/tests_nn/{}_{}'.format(dataset,trial_no)
        
    file_name = 'graph.uniform.trial{}.all.max_scale.mseloss.{}layers.nn{}.statistics.txt'.format(trial_no,n_layers,n_neighbors)
    scores = pd.read_csv(os.path.join(data_dir,file_name),sep='\t').values[:,2].astype(float)

    scores[np.isinf(scores)] = 10**10
    eval_df['{}.trial{}'.format(method,trial_no)] = scores

Trial 1


FileNotFoundError: [Errno 2] No such file or directory: '/data/cb/alexwu/mm_finemap/results/tests_nn/share_seq_both_branches/graph.uniform.trial1.all.max_scale.mseloss.10layers.nn15.statistics.txt'

In [ ]:
for trial_no in range(1,6):

    print('Trial {}'.format(trial_no))
    
    if trial_no == 1:
        data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}'.format(dataset)
    else:
        data_dir = '/data/cb/alexwu/mm_finemap/datasets/{}_{}'.format(dataset,trial_no)

    # correlations
    with h5py.File(os.path.join(data_dir,'dataset.h5'),'r') as f:
        data_df = pd.DataFrame()
        for k in ['dist','corr','group_corr','gene_percent_cells','peak_percent_cells']:
            if k in f.keys():
                data_df[k] = f[k][:]

    small_df = data_df[abs(data_df['dist']) < distance]

    # small_df = small_df[~np.isnan(small_df['group_corr'])]
    small_df = small_df[small_df['gene_percent_cells'] > gene_percent_cells]
    small_df = small_df[small_df['peak_percent_cells'] > peak_percent_cells]
    
    eval_df['corr.trial{}'.format(trial_no)] = small_df['corr'].replace(np.nan, 0).values
    eval_df['group_corr.trial{}'.format(trial_no)] = small_df['group_corr'].replace(np.nan, 0).values
    

In [44]:
eval_df2 = eval_df[(abs(eval_df['dist']) > 500000) | (abs(eval_df['dist']) < 500)]

# hic_values = eval_df['hic'].values
# hic_values[np.isnan(hic_values)] = 0
# eval_df['hic'] = hic_values

eval_df3 = eval_df[~np.isnan(eval_df['hic.new2'])]
eval_df4 = eval_df3[abs(eval_df3['dist']) > 10000]

eval_df5 = eval_df[~np.isnan(eval_df['eqtl.all'])]
eval_df5 = eval_df5[(eval_df5['eqtl.all'] < 1e-10) | (eval_df5['eqtl.all'] > 0.9)]

# method_list = ['graph.trial{}'.format(trial_no) for trial_no in range(1,4)] + \
#     ['GVAR.trial{}'.format(trial_no) for trial_no in range(1,4)] + ['granger','group_corr','corr','ABC.score'] 